In [10]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)


DATA_ROOT = '../data'

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [13]:
# Read in the data
df_db = pd.read_csv(f'{DATA_ROOT}/raw/DS_train/DS_train(2020-06--2022-06-01).csv', engine="pyarrow", delimiter='\t')
df_db.shape

(69173886, 4)

In [14]:
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69173886 entries, 0 to 69173885
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   WebPriceId    int64         
 1   DateObserve   datetime64[ns]
 2   StockStatus   object        
 3   CurrentPrice  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 2.1+ GB


In [ ]:
def select_data_for_date(df: pd.DataFrame, date: str):
    